Based on https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb

In [ ]:
! pip install datasets transformers seqeval corus razdel -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.9 MB/s eta 0:00

In this notebook we will finetune model for token classification problem (Named Entity Recognition). For speed it up we will use tiny BERT model.


This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
model_checkpoint = "huawei-noah/TinyBERT_General_4L_312D"
batch_size = 16

## Loading the dataset

We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

For our example here, we'll use the CONLL 2003 dataset.

In [ ]:
from datasets import load_dataset, load_metric

datasets = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Example of data:

In [ ]:
datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the features of the dataset:

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:

* 'PER' for person
* 'ORG' for organization
* 'LOC' for location
* 'MISC' for miscellaneous

In [ ]:
datasets["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))


show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,3890,"[Toronto, 4, CHICAGO, 2]","[NNP, CD, NNP, CD]","[B-NP, I-NP, I-NP, I-NP]","[B-ORG, O, B-ORG, O]"
1,6590,"[Collated, results, of, African, Nations, Cup, preliminary, round, ,, second, leg, matches, played, at, the, weekend, :]","[VBN, NNS, IN, NNP, NNPS, NNP, JJ, NN, ,, JJ, NN, NNS, VBD, IN, DT, NN, :]","[B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, B-VP, B-PP, B-NP, I-NP, O]","[O, O, O, B-MISC, I-MISC, I-MISC, O, O, O, O, O, O, O, O, O, O, O]"
2,3633,"[Ross, County, 1, Queen, 's, Park, 2]","[NNP, NNP, CD, NNP, POS, NNP, CD]","[B-NP, I-NP, I-NP, I-NP, B-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, O]"
3,3335,"[In, June, ,, Promodes, signed, an, outline, agreement, to, sell, its, Specia, unit, --, which, runs, 100, Dia, stores, in, France, --, to, Germany, 's, Aldi, .]","[IN, NNP, ,, NNP, VBD, DT, NN, NN, TO, VB, PRP$, NNP, NN, :, WDT, VBZ, CD, NNP, NNS, IN, NNP, :, TO, NNP, POS, NNP, .]","[B-PP, B-NP, O, B-NP, B-VP, B-NP, I-NP, I-NP, B-VP, I-VP, B-NP, I-NP, I-NP, O, B-NP, B-VP, B-NP, I-NP, I-NP, B-PP, B-NP, O, B-PP, B-NP, B-NP, I-NP, O]","[O, O, O, B-ORG, O, O, O, O, O, O, O, B-ORG, O, O, O, O, O, B-ORG, O, O, B-LOC, O, O, B-LOC, O, B-ORG, O]"
4,13408,"[Egypt, police, catch, ancient, manuscript, thieves, .]","[NNP, NN, NN, JJ, NN, NNS, .]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, O]","[B-LOC, O, O, O, O, O, O]"
5,12757,"[In, the, past, ,, Mobutu, has, been, elected, without, opposition, .]","[IN, DT, NN, ,, NNP, VBZ, VBN, VBN, IN, NN, .]","[B-PP, B-NP, I-NP, O, B-NP, B-VP, I-VP, I-VP, B-PP, B-NP, O]","[O, O, O, O, B-PER, O, O, O, O, O, O]"
6,13623,"[Repsol, shares, up, 65, pesetas, on, H1, results, .]","[JJ, NNS, RB, CD, NNS, IN, NN, NNS, .]","[B-NP, I-NP, B-ADVP, B-NP, I-NP, B-PP, B-NP, I-NP, O]","[B-ORG, O, O, O, O, O, O, O, O]"
7,10531,"[The, loss, of, Sosa, ,, who, appeared, in, all, 124, games, this, season, ,, is, a, huge, blow, to, the, Cubs, ', playoff, hopes, .]","[DT, NN, IN, NNP, ,, WP, VBD, IN, DT, CD, NNS, DT, NN, ,, VBZ, DT, JJ, NN, TO, DT, NNP, POS, NN, NNS, .]","[B-NP, I-NP, B-PP, B-NP, O, B-NP, B-VP, B-PP, B-NP, I-NP, I-NP, B-NP, I-NP, O, B-VP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-NP, I-NP, I-NP, O]","[O, O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, O, O, O, O]"
8,12874,"[Workers, at, Serbia, 's, Zastava, arms, factory, entered, the, ninth, day, of, their, protest, over, unpaid, wages, on, Thursday, with, management, accused, them, of, rejecting, talks, .]","[NNS, IN, NNP, POS, NNP, NNS, NN, VBD, DT, JJ, NN, IN, PRP$, NN, IN, JJ, NNS, IN, NNP, IN, NN, VBD, PRP, IN, VBG, NNS, .]","[B-NP, B-PP, B-NP, B-NP, I-NP, I-NP, I-NP, B-VP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, B-PP, B-NP, B-VP, B-NP, B-PP, B-VP, B-NP, O]","[O, O, B-LOC, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,12171,"[(, Note, -, contents, are, displayed, in, reverse, order, to, that, in, the, printed, Journal, )]","[(, VB, :, NNS, VBP, VBN, IN, NN, NN, TO, DT, IN, DT, VBN, NNP, )]","[O, B-VP, O, B-NP, B-VP, I-VP, B-PP, B-NP, I-NP, B-PP, B-NP, B-PP, B-NP, I-NP, I-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, O]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-200 if i is None else example["ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


Here we set the labels of all special tokens to -200 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -200 to the other subtokens from the same word. We propose the two strategies here, just change the flag `label_all_tokens`.

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples, data_column_name="ner_tags", label_all_tokens=True, label_list=None):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[data_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        if label_list is not None:
          label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_list = datasets["train"].features[f"ner_tags"].feature.names

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForTokenClassification: ['fit_denses.4.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'fit_denses.1.bias', 'fit_denses.3.bias', 'fit_denses.0.bias', 'cls.seq_relationship.bias', 'fit_denses.2.bias', 'fit_denses.0.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.2.weight', 'fit_denses.3.weight', 'fit_denses.1.weight', 'fit_denses.4.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model th

In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elem

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

<ipython-input-21-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [ ]:
example = datasets['train'][0]
labels = example['ner_tags']
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -200

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.2182376384735107,
 'eval_precision': 0.01919212736359421,
 'eval_recall': 0.0884168051269879,
 'eval_f1': 0.03153839641012615,
 'eval_accuracy': 0.059250051792003315,
 'eval_runtime': 13.0828,
 'eval_samples_per_second': 263.934,
 'eval_steps_per_second': 16.51}

At the beginning of training, let's freeze all parameters in the model except the last layer and see how well it learns.

In [ ]:
for param in model.bert.parameters():
    param.requires_grad = False

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[-0.0211, -0.0190,  0.0203,  ..., -0.0072,  0.0210, -0.0231],
        [-0.0028,  0.0146, -0.0215,  ...,  0.0092, -0.0148,  0.0053],
        [-0.0255,  0.0105, -0.0130,  ..., -0.0133, -0.0062, -0.0163],
        ...,
        [-0.0355, -0.0123, -0.0225,  ..., -0.0020,  0.0058, -0.0361],
        [-0.0037,  0.0216, -0.0276,  ..., -0.0196, -0.0039, -0.0566],
        [ 0.0092, -0.0130,  0.0082,  ...,  0.0002, -0.0156,  0.0154]],
       device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


We can now finetune our model by just calling the `train` method:

In [ ]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.056700,1.619745,0.229050,0.014598,0.027446,0.787446
2,1.471200,1.208843,0.426357,0.013055,0.025334,0.787480
3,1.092800,0.955340,0.450346,0.023143,0.044023,0.790829
4,0.878300,0.809819,0.429372,0.045455,0.082206,0.798943
5,0.809200,0.727599,0.456291,0.081771,0.138688,0.810338
6,0.728200,0.675608,0.478301,0.122953,0.195619,0.822146
7,0.677500,0.640766,0.493089,0.156658,0.237774,0.832142
8,0.643700,0.618077,0.501143,0.182174,0.267212,0.838979
9,0.633900,0.605097,0.508942,0.199264,0.286397,0.843329
10,0.626300,0.600827,0.512722,0.205673,0.293580,0.844659


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=8780, training_loss=0.9499539577336409, metrics={'train_runtime': 128.5973, 'train_samples_per_second': 1091.858, 'train_steps_per_second': 68.275, 'total_flos': 183084744356064.0, 'train_loss': 0.9499539577336409, 'epoch': 10.0})

The model is undertrained: it looks like more layers need to be trained. Let's unfreeze them all (but perhaps it would be more correct to unfreeze just a few of the top ones), and learn another 20 epochs.

In [ ]:
# unfreeze
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.300400,0.198167,0.724235,0.733088,0.728635,0.943737
2,0.183700,0.169536,0.760617,0.784358,0.772305,0.950901
3,0.146200,0.150335,0.802733,0.801685,0.802209,0.956944
4,0.122200,0.149102,0.795771,0.812960,0.804274,0.957928
5,0.105400,0.148640,0.801526,0.822929,0.812086,0.958584
6,0.094600,0.148480,0.804193,0.828626,0.816226,0.959343
7,0.085700,0.150129,0.817897,0.836340,0.827016,0.961311
8,0.077400,0.156585,0.804174,0.836815,0.820170,0.958739
9,0.071100,0.149575,0.825324,0.837764,0.831498,0.961829
10,0.064700,0.155605,0.824451,0.837171,0.830762,0.962157


TrainOutput(global_step=17560, training_loss=0.08562984113541171, metrics={'train_runtime': 480.2616, 'train_samples_per_second': 584.723, 'train_steps_per_second': 36.563, 'total_flos': 365889397149108.0, 'train_loss': 0.08562984113541171, 'epoch': 20.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.16759346425533295,
 'eval_precision': 0.8288109933620589,
 'eval_recall': 0.8446475195822454,
 'eval_f1': 0.8366543231646387,
 'eval_accuracy': 0.9633139976520958,
 'eval_runtime': 4.5487,
 'eval_samples_per_second': 759.126,
 'eval_steps_per_second': 47.487,
 'epoch': 20.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.919008875739645,
  'recall': 0.9491978609625669,
  'f1': 0.9338594513340849,
  'number': 2618},
 'MISC': {'precision': 0.8101694915254237,
  'recall': 0.776604386677498,
  'f1': 0.7930319369556201,
  'number': 1231},
 'ORG': {'precision': 0.8330078125,
  'recall': 0.8297665369649806,
  'f1': 0.8313840155945419,
  'number': 2056},
 'PER': {'precision': 0.9108847867600255,
  'recall': 0.9433091628213579,
  'f1': 0.9268134715025907,
  'number': 3034},
 'overall_precision': 0.8826316949526118,
 'overall_recall': 0.8959615169482045,
 'overall_f1': 0.8892466551934715,
 'overall_accuracy': 0.9733744261045006}

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-PER,I-PER,B-ORG,I-ORG,B-LOC,I-LOC,B-MISC,I-MISC
O,49333,38,9,66,79,34,15,64,34
B-PER,43,2898,19,42,5,20,0,7,0
I-PER,60,60,2623,12,7,3,0,0,0
B-ORG,101,83,2,1754,19,55,1,40,1
I-ORG,136,1,4,33,684,16,13,1,12
B-LOC,26,19,0,37,1,2498,17,18,2
I-LOC,16,0,0,0,14,12,235,0,4
B-MISC,114,21,0,43,1,42,0,991,19
I-MISC,63,1,7,3,16,2,12,31,255


In [ ]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

# Model Inference

In [ ]:
import torch

In [ ]:

text = ' '.join(datasets['test'][4]['tokens'])
text

'But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .'

In [ ]:
tokens = tokenizer(text, return_tensors='pt', padding="max_length").to(model.device)

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


torch.Size([1, 29, 9])

In [ ]:
pred.logits[0][-2]

tensor([ 6.7237, -4.5192, -4.5604, -4.7529, -4.6183, -4.0489, -5.2372, -4.0128,
        -5.0658], device='cuda:0')

In [ ]:
pred.logits

tensor([[[ 5.1953, -2.5026, -3.2566, -3.4644, -3.9706, -2.4998, -4.2496,
          -3.3329, -4.3698],
         [ 6.7017, -3.3649, -4.7581, -4.2649, -5.4764, -4.1611, -5.9670,
          -4.9732, -5.6547],
         [-1.0853, -0.2614, -0.9639, -0.4344, -1.5228,  7.2038,  0.4973,
          -0.2458, -2.3834],
         [ 7.7511, -4.4218, -4.6065, -5.5468, -5.4963, -5.1720, -6.0952,
          -5.9658, -5.9133],
         [ 7.5643, -4.5319, -5.1651, -4.4686, -5.2365, -5.3582, -6.3704,
          -5.6010, -5.9780],
         [ 7.2790, -3.4277, -4.7348, -5.3204, -5.3646, -5.1979, -6.9373,
          -5.1272, -5.7711],
         [ 7.8795, -4.2608, -4.9145, -5.7924, -5.7540, -4.9108, -6.5371,
          -5.8660, -6.1181],
         [ 7.3678, -3.8215, -5.4056, -4.3520, -4.8866, -5.5665, -5.8927,
          -5.3535, -5.3525],
         [ 8.1760, -4.8475, -5.2845, -6.1399, -5.3183, -5.8592, -5.6889,
          -6.0015, -5.1860],
         [ 8.0911, -5.0998, -5.3644, -5.8139, -5.3953, -5.6922, -5.6749,
         

In [ ]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
but             O         
china           B-LOC     
saw             O         
their           O         
luck            O         
desert          O         
them            O         
in              O         
the             O         
second          O         
match           O         
of              O         
the             O         
group           O         
,               O         
crashing        O         
to              O         
a               O         
surprise        O         
2               O         
-               O         
0               O         
defeat          O         
to              O         
newcomers       O         
uzbekistan      B-LOC     
.               O         
[SEP]           O         


A simpler application of the model: Pipeline by huggingface

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [ ]:
print(text)
print(pipe(text))

But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .
[{'entity_group': 'LABEL_0', 'score': 0.9998895, 'word': 'but', 'start': 0, 'end': 3}, {'entity_group': 'LABEL_5', 'score': 0.99638844, 'word': 'china', 'start': 4, 'end': 9}, {'entity_group': 'LABEL_0', 'score': 0.99997103, 'word': 'saw their luck desert them in the second match of the group, crashing to a surprise 2 - 0 defeat to newcomers', 'start': 10, 'end': 118}, {'entity_group': 'LABEL_5', 'score': 0.99520457, 'word': 'uzbekistan', 'start': 119, 'end': 129}, {'entity_group': 'LABEL_0', 'score': 0.99989545, 'word': '.', 'start': 130, 'end': 131}]


# Full pipeline again 

Lets try to run same pipeline for other model and dataset. Now, on Russian dataset RuDReC

In [ ]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2023-04-18 08:37:21--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-04-18 08:37:21--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.02s   

2023-04-18 08:37:22 (81.8 MB/s) - ‘rudrec_annotated.json’ saved [1773014/1773014]



In [ ]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


Let's see what the entities are: medicines, form of medicines, class of medicines, indications for use, side effects, and other illnesses/symptoms.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Reference to the brand name of the medicine or product
ingredients/active ingredients.
* **DRUGCLASS** Mentions of classes of drugs such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablets
or liquid that describes the physical form in which
the drug will be delivered to the patient.
* **DI** Any indication/symptom that identifies a reason for
to administer/ prescribe the medication.
* **ADR** Mention of undesirable medical events that occur as
a consequence of the medication and are not related to
symptoms of the treatment.
* **FINDING** Any ARI or ADR that has not been directly experienced by
the patient or his/her family members, or associated with
medical history/drug labeling or any disease entities if
the annotator is unclear about the type


Let's write a function to move the entity markup to the word level. We will use [IOB](https://en.wikipedia.org/wiki/Inside-outside-beginning_(tagging))-notation to separate multiple entities of the same type going in a row.


In [ ]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}


print(extract_labels(drugs[0]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

Let's put all kinds of tags together in a list.

In [ ]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Now, lets compile dataset to the same format, as we seen for CoNLL with HuggingFace Datasets

In [ ]:
from datasets import Dataset, DatasetDict

ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3847
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 962
    })
})

In [ ]:
tokenized_datasets = ner_data.map(
    lambda example: tokenize_and_align_labels(example, data_column_name='tags', label_list=label_list),
    batched=True,
)

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels=len(label_list))
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized 

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.779035,0.000000,0.000000,0.000000,0.838155
2,No log,0.745753,0.000000,0.000000,0.000000,0.838155
3,0.836500,0.731410,0.250000,0.000389,0.000776,0.838204
4,0.836500,0.693314,0.550633,0.022545,0.043316,0.840541
5,0.684600,0.667101,0.577830,0.063488,0.114406,0.844463
6,0.684600,0.647502,0.556361,0.109355,0.182783,0.848140
7,0.629300,0.633818,0.623137,0.135398,0.222459,0.851605
8,0.629300,0.624423,0.638130,0.157424,0.252546,0.853484
9,0.594500,0.609478,0.594705,0.189168,0.287034,0.855233
10,0.594500,0.602064,0.614781,0.192926,0.293688,0.856442


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=4820, training_loss=0.6034910811428212, metrics={'train_runtime': 262.8814, 'train_samples_per_second': 292.68, 'train_steps_per_second': 18.335, 'total_flos': 178055208859110.0, 'train_loss': 0.6034910811428212, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5660775899887085,
 'eval_precision': 0.6071638285378743,
 'eval_recall': 0.2679450634879502,
 'eval_f1': 0.37180870190578924,
 'eval_accuracy': 0.862767209256717,
 'eval_runtime': 2.2836,
 'eval_samples_per_second': 421.273,
 'eval_steps_per_second': 26.713,
 'epoch': 20.0}

# Self-assessment

Try to raise F1 on a test sample. 

What can be tried:
* Change training parameters (number of epochs, learning rate, etc.)
* Put other models (including larger ones) at initialisation
* Mark the subword tokens differently (e.g. label_all_tokens=True)
* Augment the data